In [2]:
import duckdb
con = duckdb.connect(database=':memory:')

### Automatically create tables

In [3]:
def get_query(input_db, tablename):
    feautures = []
    inputs_clean= [i for i in input_db.replace("\n", "").split("|") if i != ""]
    values = []
    k = 0
    for i in range(int(len(inputs_clean)//2)):
        values.append(inputs_clean[k] + inputs_clean[k+1] + ",  ")
        feautures.append(inputs_clean[k])
        k += 2
    query = "CREATE TABLE IF NOT EXISTS " + str(tablename) + "("  +     "".join(values)+ ")"
    return query, feautures

def get_inser_query(input_values, tablename,feautures ):
    nr_of_features = len(feautures)
    query_lst = []
    k = 0
    values_clean =  [i for i in input_values.replace("\n", "").split("|") if i != ""]
    for i in range(int(len(values_clean)//nr_of_features)):
        query_lst.append( "(" + "".join(["'" + str(v).replace(" ", "") +"'" + "," for v in values_clean[k:k+nr_of_features]]) + ")")
        k += nr_of_features
        insert_query = "INSERT INTO " + tablename + "(" + ",".join(feautures) + ")" +  "   VALUES "+",".join(query_lst) + ";"   
    return insert_query

def make_and_fill_table(tablename):
    global input_values
    global input_db
    query , feautures = get_query(input_db, tablename)
    con.execute(query)
    con.execute(get_inser_query(input_values, tablename,feautures ))

# 1699. Number of Calls Between Two Persons

In [2]:
con.execute('''
CREATE TABLE IF NOT EXISTS Calls (
    from_id int NOT NULL,
    to_id int NOT NULL,
    duration int NOT NULL
    )
''')

con.execute('''
INSERT INTO Calls (from_id,  to_id, duration)
VALUES
(1  ,2     , 59   ),
(2  ,1     , 11   ),
(1  ,3     , 20   ),
(3  ,4     , 100  ),
(3  ,4     , 200  ),
(3  ,4     , 200  ),
(4  ,3     , 499  ),; ''')

In [3]:
con.execute("""
SELECT person1, person2, sum(calls) AS call_count,  sum(duration) as total_duration
FROM 
( select 
    from_id as person1, 
    to_id as person2, 
    1 as calls, 
    duration
    from Calls
    where from_id < to_id
    union all
    select 
    to_id as person1, 
    from_id as person2, 
    1 as calls, 
    duration
    from Calls
    where to_id < from_id)  c
GROUP BY person1, person2
""").fetchdf()

,person1,person2,call_count,total_duration
0,1,2,2.0,70.0
1,1,3,1.0,20.0
2,3,4,4.0,999.0


# 1445. Apples & Oranges

In [22]:
con.execute('''
CREATE TABLE IF NOT EXISTS Sales (
    sale_date date NOT NULL,
    fruit varchar(32) NOT NULL,
    sold_num int NOT NULL
    )
''')

con.execute('''
INSERT INTO Sales (sale_date,  fruit, sold_num)
VALUES
('2020-05-01'   , 'apples'    , 10   ),
('2020-05-01 ', 'oranges', 8),
(' 2020-05-02  ', 'apples', 15),
('2020-05-02 ', 'oranges', 15),


('2020-05-03 ', 'apples', 20),
('2020-05-03 ', 'oranges', 0),
('2020-05-04 ', 'apples', 15),
('2020-05-04 ', 'oranges', 16)


; ''')

In [78]:
con.execute("""
SELECT sale_date, sum(sold_num) as diff
FROM (
SELECT  sale_date, count(fruit) as fruit, sum(sold_num) as sold_num
FROM Sales
WHERE fruit = 'apples'
GROUP BY sale_date

union 
SELECT  sale_date, count(fruit) as fruit, -sum(sold_num) as sold_num
FROM Sales
WHERE fruit = 'oranges'
GROUP BY sale_date) c

GROUP BY sale_date

""").fetchdf()

,sale_date,diff
0,2020-05-01,2.0
1,2020-05-02,0.0
2,2020-05-03,20.0
3,2020-05-04,-1.0


# 1251. Average Selling Price

In [89]:
con.execute('''
CREATE TABLE IF NOT EXISTS Prices (
    product_id int NOT NULL,
    start_date date NOT NULL,
    end_date date NOT NULL,
    price int NOT NULL
    )
''')

con.execute('''
CREATE TABLE IF NOT EXISTS UnitsSold (
    product_id int NOT NULL,
    purchase_date date NOT NULL,
    units int NOT NULL
    )
''')

con.execute('''
INSERT INTO Prices (product_id,  start_date, end_date, price)
VALUES
(1,  '2019-02-17', '2019-02-28',  5    ),
(1,  '2019-03-01', '2019-03-22',  20    ),
(2,  '2019-02-01', '2019-02-20',  15   ),
(2,  '2019-02-21', '2019-03-31',  30    )

; ''')


con.execute('''
INSERT INTO UnitsSold (product_id,  purchase_date, units)
VALUES
(1, '2019-02-25',  100    ),
(1,  '2019-03-01',  15    ),
(2,  '2019-02-10',  200  ),
(2,   '2019-03-22',  30    )

; ''')

In [129]:
con.execute("""
SELECT product_id, ROUND(sum(income) / sum(units),2) as 'average_price'
FROM(
SELECT  UnitsSold.product_id, (UnitsSold.units * Prices.price) AS 'income' , units
FROM Prices
JOIN UnitsSold ON Prices.product_id = UnitsSold.product_id WHERE purchase_date <= end_date and purchase_date>=start_date) c
GROUP BY product_id

""").fetchdf()

,product_id,average_price
0,1,6.0
1,2,16.0


# 1571. Warehouse Manager

In [150]:
con.execute('''
CREATE TABLE IF NOT EXISTS Warehouse (
    name varchar NOT NULL,
    product_id int NOT NULL,
    units int NOT NULL
    )
''')

con.execute('''
CREATE TABLE IF NOT EXISTS Products (
    product_id int NOT NULL,
    product_name varchar NOT NULL,
    Width int NOT NULL,
    Length int NOT NULL,
    Height int NOT NULL
    )
''')

con.execute('''
INSERT INTO Warehouse (name,  product_id, units)
VALUES
('LCHouse1', 1,1),
('LCHouse1', 2,10),
('LCHouse1', 3,5),
('LCHouse2', 1,2),
('LCHouse2', 2,2),
('LCHouse3', 4,1)




; ''')


con.execute('''
INSERT INTO Products (product_id,  product_name, Width,Length, Height )
VALUES
(1, 'LC-TV', 5,50,40),
(2, 'LC-KeyChain', 5,5,5),
(3, 'LC-Phone', 2,10,10),
(4, 'LC-T-Shirt', 4,10,20)

; ''')

In [169]:
con.execute("""
SELECT name as warehouse_name, sum(Width*Length*Height* units) as volume
FROM Warehouse 
JOIN Products on Products.product_id = Warehouse.product_id
GROUP BY name

""").fetchdf()

,warehouse_name,volume
0,LCHouse1,12250.0
1,LCHouse2,20250.0
2,LCHouse3,800.0


# 1633. Percentage of Users Attended a Contest

In [2]:
con.execute('''
CREATE TABLE IF NOT EXISTS Users (
    user_id int NOT NULL,
    user_name varchar NOT NULL
    )
''')

con.execute('''
CREATE TABLE IF NOT EXISTS Register (
    contest_id int NOT NULL,
    user_id int NOT NULL,
   
    )
''')

con.execute('''
INSERT INTO Users (user_id,  user_name)
VALUES
(6, 'Alice'),
(2,'Bob'),
(7, 'Alex')



; ''')


con.execute('''
INSERT INTO Register (contest_id, user_id  )
VALUES
(215,6),
(209,2),
(208,2),
(210,6),
(208,6),
(209,7),
(209,6),
(215,7),
(208,7),
(210,2),
(207,2),
(210,7)

; ''')

In [53]:
con.execute("""
SELECT contest_id, ROUND(count(user_id) /  (SELECT count(user_id)
FROM Users ) *100,2) as percentage
FROM Register
GROUP BY contest_id
ORDER BY percentage DESC, contest_id ASC

""").fetchdf()

,contest_id,percentage
0,208,100.0
1,209,100.0
2,210,100.0
3,207,0.0
4,215,0.0


# 1173. Immediate Food Delivery I

In [55]:
con.execute('''
CREATE TABLE IF NOT EXISTS Delivery (
    delivery_id int NOT NULL,
    customer_id int NOT NULL,
    order_date date,
    customer_pref_delivery_date date
    )
''')


con.execute('''
INSERT INTO Delivery (delivery_id, customer_id  , order_date, customer_pref_delivery_date)
VALUES
(1,1, '2019-08-01', '2019-08-02'),
(2,1, '2019-08-01', '2019-08-01'),
(3,2, '2019-07-01', '2019-08-02'),

; ''')

In [63]:
con.execute("""
SELECT  ROUND(count(customer_id) / (SELECT  count(customer_id) 
FROM Delivery)*100,2) as immediate_percentage
FROM Delivery
WHERE order_date = customer_pref_delivery_date
""").fetchdf()

,immediate_percentage
0,0.0


# 1211. Queries Quality and Percentage

In [210]:
input_db = """
| query_name  | varchar |
| result      | varchar |
| position    | int     |
| rating      | int     |
"""


input_values = """
| Dog        | Golden Retriever  | 1        | 5      |
| Dog        | German Shepherd   | 2        | 5      |
| Dog        | Mule              | 200      | 1      |
| Cat        | Shirazi           | 5        | 2      |
| Cat        | Siamese           | 3        | 3      |
| Cat        | Sphynx            | 7        | 4      |
"""

In [214]:
make_and_fill_table("Queries")

In [275]:
con.execute("""
SELECT  query_name, ROUND(avg(rating/position),2) as quality, round((sum(if(rating <3, 1, 0))/ count(*))*100,2) AS poor_query_percentage
FROM Queries
GROUP BY query_name

""").fetchdf()

,query_name,quality,poor_query_percentage
0,Dog,2.33,0.0
1,Cat,0.33,0.0


# 1193. Monthly Transactions I

In [276]:
input_db = """
| id            | int     |
| country       | varchar |
| state         | varchar    |
| amount        | int     |
| trans_date    | date    |

"""


input_values = """
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |

"""

make_and_fill_table("Transactions")

In [341]:
con.execute("""
SELECT  DATE_FORMAT(trans_date, '%Y-%m') as month, country, 
COUNT(state) AS trans_count,
COUNT(IF(state = 'approved', 1, NULL)) AS approved_count,
SUM(amount) AS trans_total_amount,
SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) as approved_total_amount


FROM Transactions
GROUP BY month, country

""").fetchdf()

,country,approved_count
0,US,3000.0
1,DE,2000.0


# 1607. Sellers With No Sales

In [4]:
input_db = """
| customer_id   | int     |
| customer_name | varchar |

"""


input_values = """
| 101          | Alice         |
| 102          | Bob           |
| 103          | Charlie       |

"""

make_and_fill_table("Customer")

In [ ]:
input_db = """
| customer_id   | int     |
| customer_name | varchar |

"""


input_values = """
| 101          | Alice         |
| 102          | Bob           |
| 103          | Charlie       |

"""

make_and_fill_table("Customer")


In [6]:
input_db = """
| order_id      | int     |
| sale_date     | date    |
| order_cost    | int     |
| customer_id   | int     |
| seller_id     | int     |

"""


input_values = """
| 1           | 2020-03-01 | 1500         | 101         | 1           |
| 2           | 2020-05-25 | 2400         | 102         | 2           |
| 3           | 2019-05-25 | 800          | 101         | 3           |
| 4           | 2020-09-13 | 1000         | 103         | 2           |
| 5           | 2019-02-11 | 700          | 101         | 2           |
"""

make_and_fill_table("Orders")


In [8]:
input_db = """
| seller_id     | int     |
| seller_name   | varchar |

"""


input_values = """
| 1           | Daniel      |
| 2           | Elizabeth   |
| 3           | Frank       |
"""

make_and_fill_table("Seller")

In [44]:
con.execute("""
(SELECT DISTINCT seller_name
FROM Seller)
EXCEPT 
(SELECT DISTINCT seller_name
FROM Orders
JOIN Seller ON Seller.seller_id =  Orders.seller_id
WHERE YEAR(Orders.sale_date)  = 2020)

""").fetchdf()

,seller_name
0,Frank


# 619. Biggest Single Number


In [76]:
import duckdb
con = duckdb.connect(database=':memory:')

input_db = """
| num         | int  |

"""


input_values = """
| 8   |
| 8   |
| 3   |
| 3   |




"""

make_and_fill_table("MyNumbers")

In [82]:
con.execute("""
SELECT (CASE WHEN num > 0 THEN num ELSE 0 END) as num
FROM 
(SELECT NUM 
FROM 
(SELECT num, count(num) as amount
FROM MyNumbers
GROUP BY num
HAVING amount = 1
ORDER BY num DESC) LIMIT 1)
""").fetchdf()

,num
